# Iterative Prompt Development
In this lesson, you'll iteratively analyze and refine your prompts to generate marketing copy from a product fact sheet.

## Setup

In [1]:
pip install openai python-dotenv

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from openai import AzureOpenAI
import os

os.environ["AZURE_OPENAI_API_KEY"]="" # TODO copy your AZURE OPENAI KEY
os.environ["AZURE_OPENAI_ENDPOINT"]="" # TODO copy your AZURE OPENAI ENDPOINT URL
os.environ["AZURE_DEPLOYMENT_NAME"]="" # TODO copy your Azure OPENAI DEPLOYMENT NAME


os.environ["AZURE_OPENAI_API_KEY"]="1d0e668fdc424a04af2124483f1a63dc" # TODO copy your AZURE OPENAI KEY
os.environ["AZURE_OPENAI_ENDPOINT"]="https://uksouth.api.cognitive.microsoft.com/" # TODO copy your AZURE OPENAI ENDPOINT URL
os.environ["AZURE_DEPLOYMENT_NAME"]="aifashionassistant-gpt-35-1106" # TODO copy your Azure OPENAI DEPLOYMENT NAME


client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),    
  api_version="2023-12-01-preview",
)

In [4]:
def get_completion(client, prompt):
    messages = [{"role": "user", "content": prompt}]
    
    response = client.chat.completions.create(
        model=os.getenv("AZURE_DEPLOYMENT_NAME"),  
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )

    return response.choices[0].message.content

**Note**: In June 2023, OpenAI updated gpt-3.5-turbo. The results you see in the notebook may be slightly different than those in the video. Some of the prompts have also been slightly modified to product the desired results.

## Generate a marketing product description from a product fact sheet

In [5]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [6]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(client, prompt)
print(response)


Introducing the SWC-100 and SWC-110 office chair, part of our stunning mid-century inspired furniture collection. Available in a variety of shell colors and base finishes, this chair is the perfect addition to any home or business setting. Choose from plastic back and front upholstery or full upholstery in a range of fabric and leather options. With options for armrests and different seat foam densities, you can customize this chair to fit your specific needs. 

Constructed with a 5-wheel plastic coated aluminum base and featuring pneumatic chair adjust for easy raise/lower action, this chair is as functional as it is stylish. The dimensions of the chair make it suitable for a variety of spaces, and with options for soft or hard-floor casters, you can ensure it fits seamlessly into your environment. 

Made with high-quality materials, including cast aluminum with modified nylon coating and HD36 foam, this chair is built to last. Plus, with its origin in Italy, you can trust that you ar

## Issue 1: The text is too long 
- Limit the number of words/sentences/characters.

In [7]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(client, prompt)
print(response)


Introducing the versatile and stylish SWC-100 office chair, part of our mid-century inspired furniture collection. Available in a range of colors and finishes, with options for upholstery and armrests. Designed for both home and business use, with a durable aluminum base and adjustable pneumatic lift. Made in Italy.


In [8]:
len(response.split())

48

## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [9]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(client, prompt)
print(response)

Introducing our mid-century inspired office chair, available in a variety of shell colors and base finishes. Constructed with a 5-wheel plastic coated aluminum base and pneumatic chair adjust for easy raise/lower action. Made with high-quality materials including cast aluminum, modified nylon, and HD36 foam. Designed for both home and business settings. Made in Italy.


In [10]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(client, prompt)
print(response)

Introducing the versatile and stylish mid-century inspired office chair, available in a variety of shell colors and base finishes. Constructed with a 5-wheel plastic coated aluminum base and HD36 foam seat, this chair offers comfort and durability. Perfect for home or business use. Product IDs: SWC-100, SWC-110. Made in Italy.


## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [11]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(client, prompt)
print(response)

```html
<div>
  <p>
    Introducing the latest addition to our mid-century inspired office furniture collection. This stylish and versatile chair is perfect for both home and business settings. With a variety of shell colors, base finishes, and upholstery options to choose from, you can customize this chair to fit your unique style and needs. The 5-wheel plastic coated aluminum base and pneumatic chair adjust make for easy mobility and height adjustment. The chair is constructed with high-quality materials, including a cast aluminum shell with modified nylon coating and HD36 foam seat. Made in Italy, this chair is not only stylish but also durable and functional. Choose from soft or hard-floor caster options and two seat foam densities to create the perfect chair for your space. Product ID: SWC-100, SWC-110
  </p>
  <table>
    <caption>Product Dimensions</caption>
    <tr>
      <td>WIDTH</td>
      <td>20.87”</td>
    </tr>
    <tr>
      <td>DEPTH</td>
      <td>20.08”</td>
    </tr

## Load Python libraries to view HTML

In [12]:
from IPython.display import display, HTML

In [13]:
display(HTML(response))

WIDTH,20.87”
DEPTH,20.08”
HEIGHT,31.50”
SEAT HEIGHT,17.32”
SEAT DEPTH,16.14”


## Try experimenting on your own!